In [1]:
!pip install nltk
import nltk
from nltk.corpus import stopwords
nltk.download ('stopwords')
stopWords = list (set(stopwords.words('english')))
stopWords+=['the','and','is'] #may need to manually add'the','is',etc.

You are using pip version 9.0.1, however version 9.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import requests_oauthlib
# import webbrowser
import json 
client_key = '4O8Wirkz7dN2GNyU7pV76oaYn' # what Twitter calls Consumer Key -- fill in a string here
client_secret = 'hrGSKfJSH5WU5bg1bpWXoLFNFd7oy6tVA2LTLZyJQSTii2O7AX'
def get_tokens():
 
    oauth = requests_oauthlib.OAuth1Session(client_key, client_secret=client_secret)

    request_token_url = 'https://api.twitter.com/oauth/request_token'

    fetch_response = oauth.fetch_request_token(request_token_url)

    resource_owner_key = fetch_response.get('oauth_token')
    resource_owner_secret = fetch_response.get('oauth_token_secret')


    base_authorization_url = 'https://api.twitter.com/oauth/authorize'
    
    authorization_url = oauth.authorization_url(base_authorization_url)

    print("""Please click on the URL below, 
          click on "Authorize App", 
          and then copy the authorization code and paste it below.""")
    print(authorization_url)
    
    verifier = input('Please input the verifier>>> ')


    ## Step 3. 
    oauth = requests_oauthlib.OAuth1Session(client_key,
                              client_secret=client_secret,
                              resource_owner_key=resource_owner_key,
                              resource_owner_secret=resource_owner_secret,
                              verifier=verifier)

    access_token_url = 'https://api.twitter.com/oauth/access_token'
    oauth_tokens = oauth.fetch_access_token(access_token_url)
    
    resource_owner_key = oauth_tokens.get('oauth_token')
    resource_owner_secret = oauth_tokens.get('oauth_token_secret')

    return (client_key, client_secret, resource_owner_key, resource_owner_secret, verifier)

In [3]:
try:
    # See if you can read the credentials from the file.
    # (If you have credentials for the wrong user, or expired credentials,
    # just delete the file creds.txt
    f = open("creds.txt", 'r')
    (client_key, client_secret, resource_owner_key, resource_owner_secret,
     verifier) = json.loads(f.read())
    f.close()
except:
    # If not, you'll have to get them
    # and then save them in creds.txt
    tokens = get_tokens()
    f = open("creds.txt", 'w')
    f.write(json.dumps(tokens))
    f.close()
    (client_key, client_secret, resource_owner_key, resource_owner_secret, verifier) = tokens
protected_url = 'https://api.twitter.com/1.1/account/settings.json'
oauth = requests_oauthlib.OAuth1Session(client_key,
                        client_secret=client_secret,
                        resource_owner_key=resource_owner_key,
                        resource_owner_secret=resource_owner_secret)

response = oauth.get("https://api.twitter.com/1.1/search/tweets.json", 
              params = {'q': 'rain', 'count' : 3}) 


In [4]:
def get_next_page(ids_already_retrieved=[]):
    my_params = {'count' : 100}
    if len(ids_already_retrieved) > 0: # If we have already started the paging process
        my_params['max_id'] = min(ids_already_retrieved) - 1

    my_params['q'] = 'rain'
    r = oauth.get("https://api.twitter.com/1.1/search/tweets.json", 
                 params=my_params)

    return r.json()["statuses"]

In [5]:
collected_tweets = get_next_page()
while len(collected_tweets)<200:
    collected_tweets.append(get_next_page())


In [6]:
len(collected_tweets)

200

In [7]:
class Tweet():
    def __init__(self, tweet_dict={}):
        if 'id' in tweet_dict:
            self.id = tweet_dict['id']
        else:
            self.id = ''
        
        if 'text' in tweet_dict:
            self.text = tweet_dict['text']
        else:
            self.text = ''
    
    def __str__(self):
        return 'This tweet says'+self.text
    
    def nonstop(self):
        nsw = []
        words = self.text.split()
        for w in words:
            if w not in stopWords:
                nsw.append(w)
        return nsw
    
    def words(self):
        return self.text.split()   
    
    def wl5 (self):
        l = []
        for a in self.words:
            if len(a)>4:
                l.append(a)
        return l

tweet_inss = []

for atweet in collected_tweets:
    tweet_ins = (Tweet (atweet))
    print (tweet_ins)
    tweet_inss.append(tweet_ins)
    

This tweet saysRT @_IMBOWVYBD: ความเผ็ชร้อนของด้อมนี้ อยู่ด้อมเดียวกันพี่ยังกลัว หลักฐานแน่น ภาพประกอบเน้นๆ VDO ก็มา แถลงการณ์ 3 ภาษากระจายความเผือกอย่าง…
This tweet saysRT @cuttolipta: Bangkok in the rain
This tweet saysRT @btschsplus: [BTS V OFFICIAL]
-
RUN BTS! 2018 - Epi.46
Behind the scene 2/3
- https://t.co/hVVgvDbNo6
This tweet saysRT @FybStitchHefner: When u pray 4 rain u gotta deal wit mud only ppl that been thru it would understand.
This tweet saysRT @JamesJSChai: 30 years after Grave of the Fireflies was released, fans discovered something about the poster. When you turn brightness a…
This tweet saysSUPER  JUNIOR 슈퍼주니어 'Lo Siento (Feat. Leslie Grace)' MV
1000万回突破ー＼(^o^)／＼(^o^)／＼(^o^)／＼(^o^)／＼(^o^)／＼(^o^)／＼(^o^)／… https://t.co/IwKq6bzQOs
This tweet saysIt's now Rain Shower in Auckland. Are you really surprised?
This tweet saysCheck out what I just added to my closet on Poshmark: Lululemon Black Rain coat/Wind breaker.… https://t.co/AX8tYwjbGn
This tweet saysRT @blxcknicotine:

In [8]:
print(json.dumps(collected_tweets[0], indent = 2))
texts = []
for d in collected_tweets:
    if type(d)==dict:
        texts.append(d['text'])
    

{
  "created_at": "Tue Apr 17 03:44:13 +0000 2018",
  "id": 986087912746827776,
  "id_str": "986087912746827776",
  "text": "RT @_IMBOWVYBD: \u0e04\u0e27\u0e32\u0e21\u0e40\u0e1c\u0e47\u0e0a\u0e23\u0e49\u0e2d\u0e19\u0e02\u0e2d\u0e07\u0e14\u0e49\u0e2d\u0e21\u0e19\u0e35\u0e49 \u0e2d\u0e22\u0e39\u0e48\u0e14\u0e49\u0e2d\u0e21\u0e40\u0e14\u0e35\u0e22\u0e27\u0e01\u0e31\u0e19\u0e1e\u0e35\u0e48\u0e22\u0e31\u0e07\u0e01\u0e25\u0e31\u0e27 \u0e2b\u0e25\u0e31\u0e01\u0e10\u0e32\u0e19\u0e41\u0e19\u0e48\u0e19 \u0e20\u0e32\u0e1e\u0e1b\u0e23\u0e30\u0e01\u0e2d\u0e1a\u0e40\u0e19\u0e49\u0e19\u0e46 VDO \u0e01\u0e47\u0e21\u0e32 \u0e41\u0e16\u0e25\u0e07\u0e01\u0e32\u0e23\u0e13\u0e4c 3 \u0e20\u0e32\u0e29\u0e32\u0e01\u0e23\u0e30\u0e08\u0e32\u0e22\u0e04\u0e27\u0e32\u0e21\u0e40\u0e1c\u0e37\u0e2d\u0e01\u0e2d\u0e22\u0e48\u0e32\u0e07\u2026",
  "truncated": false,
  "entities": {
    "hashtags": [],
    "symbols": [],
    "user_mentions": [
      {
        "screen_name": "_IMBOWVYBD",
        "name": "\ud83c\udf08\ubc

In [9]:
print(texts)

['RT @_IMBOWVYBD: ความเผ็ชร้อนของด้อมนี้ อยู่ด้อมเดียวกันพี่ยังกลัว หลักฐานแน่น ภาพประกอบเน้นๆ VDO ก็มา แถลงการณ์ 3 ภาษากระจายความเผือกอย่าง…', 'RT @cuttolipta: Bangkok in the rain', 'RT @btschsplus: [BTS V OFFICIAL]\n-\nRUN BTS! 2018 - Epi.46\nBehind the scene 2/3\n- https://t.co/hVVgvDbNo6', 'RT @FybStitchHefner: When u pray 4 rain u gotta deal wit mud only ppl that been thru it would understand.', 'RT @JamesJSChai: 30 years after Grave of the Fireflies was released, fans discovered something about the poster. When you turn brightness a…', "SUPER  JUNIOR 슈퍼주니어 'Lo Siento (Feat. Leslie Grace)' MV\n1000万回突破ー＼(^o^)／＼(^o^)／＼(^o^)／＼(^o^)／＼(^o^)／＼(^o^)／＼(^o^)／… https://t.co/IwKq6bzQOs", "It's now Rain Shower in Auckland. Are you really surprised?", 'Check out what I just added to my closet on Poshmark: Lululemon Black Rain coat/Wind breaker.… https://t.co/AX8tYwjbGn', 'RT @blxcknicotine: Like rain, sometimes we must fall apart in order to fall back together', 'RT @xanndiego: ok the rain ma

In [10]:
print(tweet_inss)

[<__main__.Tweet object at 0x7f701bbf4400>, <__main__.Tweet object at 0x7f701bbf42b0>, <__main__.Tweet object at 0x7f701bbf4438>, <__main__.Tweet object at 0x7f701bbf4470>, <__main__.Tweet object at 0x7f701bbf44a8>, <__main__.Tweet object at 0x7f701bbf44e0>, <__main__.Tweet object at 0x7f701bbf4518>, <__main__.Tweet object at 0x7f701bbf4550>, <__main__.Tweet object at 0x7f701bbf4588>, <__main__.Tweet object at 0x7f701bbf45c0>, <__main__.Tweet object at 0x7f701bbf45f8>, <__main__.Tweet object at 0x7f701bbf4630>, <__main__.Tweet object at 0x7f701bbf4668>, <__main__.Tweet object at 0x7f701bbf46a0>, <__main__.Tweet object at 0x7f701bbf46d8>, <__main__.Tweet object at 0x7f701bbf4710>, <__main__.Tweet object at 0x7f701bbf4748>, <__main__.Tweet object at 0x7f701bbf4780>, <__main__.Tweet object at 0x7f701bbf47b8>, <__main__.Tweet object at 0x7f701bbf47f0>, <__main__.Tweet object at 0x7f701bbf4828>, <__main__.Tweet object at 0x7f701bbf4860>, <__main__.Tweet object at 0x7f701bbf4898>, <__main__.

In [11]:
allwlong5 = []
for t in texts:
    tl = t.split()
    for wl in tl:
        if len(wl)>5 and wl not in stopWords:
            allwlong5.append(wl)
print(allwlong5)

['@_IMBOWVYBD:', 'ความเผ็ชร้อนของด้อมนี้', 'อยู่ด้อมเดียวกันพี่ยังกลัว', 'หลักฐานแน่น', 'ภาพประกอบเน้นๆ', 'แถลงการณ์', 'ภาษากระจายความเผือกอย่าง…', '@cuttolipta:', 'Bangkok', '@btschsplus:', 'OFFICIAL]', 'Epi.46', 'Behind', 'https://t.co/hVVgvDbNo6', '@FybStitchHefner:', 'understand.', '@JamesJSChai:', 'Fireflies', 'released,', 'discovered', 'something', 'poster.', 'brightness', 'JUNIOR', 'Siento', '(Feat.', 'Leslie', "Grace)'", '1000万回突破ー＼(^o^)／＼(^o^)／＼(^o^)／＼(^o^)／＼(^o^)／＼(^o^)／＼(^o^)／…', 'https://t.co/IwKq6bzQOs', 'Shower', 'Auckland.', 'really', 'surprised?', 'closet', 'Poshmark:', 'Lululemon', 'coat/Wind', 'breaker.…', 'https://t.co/AX8tYwjbGn', '@blxcknicotine:', 'sometimes', 'together', '@xanndiego:', 'curlier', 'https://t.co/ukYWfmcmPb', '@thesecret:', 'grateful', 'birds,', 'trees,', 'grass,', 'flowers.', '@stick_fgo:', '神社とか好きな人は南阿蘇にある上色見熊野座神社には絶対に行ったほうがいい', 'マジで異世界', 'https://t.co/qgHmIRwoue', '@R_Axelay', 'いやだああああ(*ﾟﾉOﾟ)&lt;ﾊﾟﾊﾟｧｧｧｱｱｱｰｰｰｰ', '@pluie_rain', 'そうなのですか！？', '4/20（

In [12]:
ad = {}
for z in allwlong5:
    if z not in ad:
        ad[z] = 0
    ad[z]+=1
most_common_word = (sorted(ad,key = lambda x:ad[x])[-1])
print (most_common_word)

brightness


In [13]:
import requests
baseurl = 'https://itunes.apple.com/search? term = Boston'
params = {'term':most_common_word,'country':'US','media':'music'}
s = requests.get(baseurl, params = params)
song_data = s.json()
song_dict = song_data['results']
songs = []
print(song_dict[:1])

[{'wrapperType': 'track', 'kind': 'song', 'artistId': 189229615, 'collectionId': 189229284, 'trackId': 189229606, 'artistName': 'Brightness', 'collectionName': 'Dance 90s', 'trackName': 'In My Dreams', 'collectionCensoredName': 'Dance 90s', 'trackCensoredName': 'In My Dreams (Short Mix)', 'collectionArtistId': 882054, 'collectionArtistName': 'Various Artists', 'artistViewUrl': 'https://itunes.apple.com/us/artist/brightness/189229615?uo=4', 'collectionViewUrl': 'https://itunes.apple.com/us/album/in-my-dreams-short-mix/189229284?i=189229606&uo=4', 'trackViewUrl': 'https://itunes.apple.com/us/album/in-my-dreams-short-mix/189229284?i=189229606&uo=4', 'previewUrl': 'https://audio-ssl.itunes.apple.com/apple-assets-us-std-000001/Music/23/0a/12/mzm.vlualrpz.aac.p.m4a', 'artworkUrl30': 'https://is3-ssl.mzstatic.com/image/thumb/Music/v4/6f/ed/2f/6fed2fe6-d4c2-384f-441e-3b3aaed594dc/source/30x30bb.jpg', 'artworkUrl60': 'https://is3-ssl.mzstatic.com/image/thumb/Music/v4/6f/ed/2f/6fed2fe6-d4c2-384f

In [18]:
class Song():
    def __init__(self, title, artist, album, length):
        self.title = title
        self.artist = artist
        self.album = album
        self.length = length
        
    def __str__(self):
        return "This is {} by {} in{} with a length of {}".format(title, artist, album, length)

    def getlength(self):
        return self.length
    def gettitle(self):
        return self.title
    def getartist(self):
        return self.artist
    
for so in song_dict:
    title = so['trackName']
    artist = so['artistName']
    album = so['collectionName']
    length = so['trackTimeMillis']
    songs.append(Song(title, artist, album, length))  


In [19]:
print(songs)

[<__main__.Song object at 0x7f701bba6f98>, <__main__.Song object at 0x7f701bba6fd0>, <__main__.Song object at 0x7f701bbb0048>, <__main__.Song object at 0x7f701bbb0080>, <__main__.Song object at 0x7f701bbb00b8>, <__main__.Song object at 0x7f701bbb00f0>, <__main__.Song object at 0x7f701bbb0128>, <__main__.Song object at 0x7f701bbb0160>, <__main__.Song object at 0x7f701bbb0198>, <__main__.Song object at 0x7f701bbb01d0>, <__main__.Song object at 0x7f701bbb0208>, <__main__.Song object at 0x7f701bbb0240>, <__main__.Song object at 0x7f701bbb0278>, <__main__.Song object at 0x7f701bbb02b0>, <__main__.Song object at 0x7f701bbb02e8>, <__main__.Song object at 0x7f701bbb0320>, <__main__.Song object at 0x7f701bbb0358>, <__main__.Song object at 0x7f701bbb0390>, <__main__.Song object at 0x7f701bbb03c8>, <__main__.Song object at 0x7f701bbb0400>, <__main__.Song object at 0x7f701bbb0438>, <__main__.Song object at 0x7f701bbb0470>, <__main__.Song object at 0x7f701bbb04a8>, <__main__.Song object at 0x7f701b

In [20]:
import csv
file = open ('playlist.txt','w')
file.write("This is {} by {} in {} with a length of {}"+'\n')
for xs in songs:
    file.write(str(xs)+'\n')
file.close()